In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import json
import os
import scipy as sp
from collections import defaultdict
from itertools import product
import gurobipy as gb
from gurobipy import GRB
from tqdm import tqdm

repeats = json.load(open('../cleanerdata/repeats.json'))
repeats = {int(k):v for k,v in repeats.items()}

locdem = pd.read_excel('../cleanerdata/locdem.xlsx')

q = locdem['Number of pallets'].to_numpy().astype(float).tolist()
q = [0] + q
q.extend([v['dem'] for v in repeats.values()])

locs = pd.read_excel('../cleanerdata/locations.xlsx')
longs, lats = locs['long'].to_numpy().tolist(), locs['lat'].to_numpy().tolist()
longs.extend([longs[v['map']] for v in repeats.values()])
lats.extend([lats[v['map']] for v in repeats.values()])

distmat = pd.read_json('../cleanerdata/distmat.json').to_numpy()
timemat = pd.read_json('../cleanerdata/timemat.json').to_numpy()
def get(i):
    try:
        i = repeats[i]['map']
    except KeyError:
        pass
    return i

mappings = set()
for k,v in repeats.items():
    mappings.add((k, v['map']))
    mappings.add((v['map'], k))

def cost(i,j):
    if (i,j) in mappings:
        return 1e+8
    i,j = get(i), get(j)
    return distmat[i,j]

def time(i,j):
    i,j = get(i), get(j)
    return timemat[i,j]

def location(i):
    return longs[i], lats[i]

In [2]:
clusters = json.load(open('./clusters.json'))
list(map(lambda x: len(x), clusters))

[3,
 3,
 3,
 3,
 3,
 3,
 3,
 3,
 3,
 3,
 3,
 3,
 3,
 3,
 3,
 3,
 3,
 3,
 3,
 3,
 3,
 3,
 3,
 3,
 3,
 3,
 3,
 3,
 3,
 3,
 3,
 3,
 3,
 3,
 3,
 3,
 3,
 3,
 3,
 3,
 3,
 3,
 3,
 3,
 3,
 3,
 3,
 3,
 3,
 3,
 3,
 3,
 3,
 3,
 3,
 3,
 3,
 3,
 3,
 3]

In [3]:
Q = 9
iter_count = 0
primary_models = []
Ls = []

for cluster in clusters:
    Neighbours = list(sorted(cluster))
    all_nodes = [0] + Neighbours.copy()
    v = len(all_nodes)
    V = list(range(v))
    N = V.copy()
    N.remove(0)
    
    def Del(i):
        F = V.copy()
        F.remove(i)
        return F
    
    m = gb.Model()
    x = m.addVars(v,v, vtype=GRB.BINARY, name='x')
    m.setObjective(sum([cost(all_nodes[i], all_nodes[j]) * x[i,j] for i,j in product(V,V)]))
    for i in N:
        m.addConstr(sum([x[i,j] for j in Del(i)]) == 1)
    for j in N:
        m.addConstr(sum([x[i,j] for i in Del(j)]) == 1)
        
    l = m.addVar(lb=0.0, vtype=GRB.INTEGER, name='L')
    Ls.append(l)
    m.addConstr(sum([x[0,j] for j in Del(0)]) == l)
    
    u = m.addVars(V, vtype=GRB.CONTINUOUS, name='u')
    for i,j in filter(lambda tup: tup[0] != tup[1], product(N,N)):
        m.addConstr(u[i] - u[j] + Q*x[i,j] <= Q - q[j])
    for i in V:
        m.addConstr(u[i] >= q[i])
    
    primary_models.append(m)

Set parameter Username
Academic license - for non-commercial use only - expires 2023-09-12


In [4]:
for model in tqdm(primary_models):
    model.optimize();

  0%|                                                    | 0/60 [00:00<?, ?it/s]

Gurobi Optimizer version 9.5.2 build v9.5.2rc0 (mac64[rosetta2])
Thread count: 8 physical cores, 8 logical processors, using up to 8 threads
Optimize a model with 17 rows, 21 columns and 44 nonzeros
Model fingerprint: 0xf3a709f1
Variable types: 4 continuous, 17 integer (16 binary)
Coefficient statistics:
  Matrix range     [1e+00, 9e+00]
  Objective range  [3e+02, 3e+04]
  Bounds range     [1e+00, 1e+00]
  RHS range        [1e+00, 8e+00]
Found heuristic solution: objective 160610.00000
Presolve removed 5 rows and 12 columns
Presolve time: 0.00s
Presolved: 12 rows, 9 columns, 30 nonzeros
Variable types: 3 continuous, 6 integer (6 binary)
Found heuristic solution: objective 83140.000000

Root relaxation: objective 4.621500e+04, 5 iterations, 0.00 seconds (0.00 work units)

    Nodes    |    Current Node    |     Objective Bounds      |     Work
 Expl Unexpl |  Obj  Depth IntInf | Incumbent    BestBd   Gap | It/Node Time

     0     0 46215.0000    0    3 83140.0000 46215.0000  44.4%     

  3%|█▍                                          | 2/60 [00:00<00:07,  7.46it/s]

Gurobi Optimizer version 9.5.2 build v9.5.2rc0 (mac64[rosetta2])
Thread count: 8 physical cores, 8 logical processors, using up to 8 threads
Optimize a model with 17 rows, 21 columns and 44 nonzeros
Model fingerprint: 0x28dd9e5a
Variable types: 4 continuous, 17 integer (16 binary)
Coefficient statistics:
  Matrix range     [1e+00, 9e+00]
  Objective range  [7e+01, 7e+04]
  Bounds range     [1e+00, 1e+00]
  RHS range        [1e+00, 8e+00]
Found heuristic solution: objective 291205.00000
Presolve removed 5 rows and 12 columns
Presolve time: 0.00s
Presolved: 12 rows, 9 columns, 30 nonzeros
Variable types: 3 continuous, 6 integer (6 binary)
Found heuristic solution: objective 172150.00000

Root relaxation: objective 1.128257e+05, 5 iterations, 0.00 seconds (0.00 work units)

    Nodes    |    Current Node    |     Objective Bounds      |     Work
 Expl Unexpl |  Obj  Depth IntInf | Incumbent    BestBd   Gap | It/Node Time

     0     0 112825.667    0    3 172150.000 112825.667  34.5%     

  5%|██▏                                         | 3/60 [00:00<00:06,  8.16it/s]

...

 97%|█████████████████████████████████████████▌ | 58/60 [00:06<00:00,  8.43it/s]

Gurobi Optimizer version 9.5.2 build v9.5.2rc0 (mac64[rosetta2])
Thread count: 8 physical cores, 8 logical processors, using up to 8 threads
Optimize a model with 17 rows, 21 columns and 44 nonzeros
Model fingerprint: 0x47c0f594
Variable types: 4 continuous, 17 integer (16 binary)
Coefficient statistics:
  Matrix range     [1e+00, 9e+00]
  Objective range  [4e+04, 1e+05]
  Bounds range     [1e+00, 1e+00]
  RHS range        [1e+00, 8e+00]
Found heuristic solution: objective 297648.00000
Presolve removed 5 rows and 12 columns
Presolve time: 0.00s
Presolved: 12 rows, 9 columns, 30 nonzeros
Variable types: 3 continuous, 6 integer (6 binary)
Found heuristic solution: objective 201854.00000

Root relaxation: objective 1.532733e+05, 5 iterations, 0.00 seconds (0.00 work units)

    Nodes    |    Current Node    |     Objective Bounds      |     Work
 Expl Unexpl |  Obj  Depth IntInf | Incumbent    BestBd   Gap | It/Node Time

     0     0 153273.333    0    3 201854.000 153273.333  24.1%     

 98%|██████████████████████████████████████████▎| 59/60 [00:07<00:00,  7.65it/s]

Gurobi Optimizer version 9.5.2 build v9.5.2rc0 (mac64[rosetta2])
Thread count: 8 physical cores, 8 logical processors, using up to 8 threads
Optimize a model with 17 rows, 21 columns and 44 nonzeros
Model fingerprint: 0x81f87bc4
Variable types: 4 continuous, 17 integer (16 binary)
Coefficient statistics:
  Matrix range     [1e+00, 9e+00]
  Objective range  [2e+04, 1e+08]
  Bounds range     [1e+00, 1e+00]
  RHS range        [1e+00, 8e+00]
Found heuristic solution: objective 161513.00000
Presolve removed 9 rows and 14 columns
Presolve time: 0.00s
Presolved: 8 rows, 7 columns, 20 nonzeros
Variable types: 3 continuous, 4 integer (4 binary)
Found heuristic solution: objective 102328.00000

Root relaxation: cutoff, 0 iterations, 0.00 seconds (0.00 work units)

Explored 1 nodes (0 simplex iterations) in 0.07 seconds (0.00 work units)
Thread count was 8 (of 8 available processors)

Solution count 2: 102328 161513 

Optimal solution found (tolerance 1.00e-04)
Best objective 1.023280000000e+05, 

100%|███████████████████████████████████████████| 60/60 [00:07<00:00,  8.37it/s]


In [5]:
model_solutions = {}
for i,model in enumerate(primary_models):
    soln = json.loads(model.getJSONSolution())
    model_solutions[i] = soln
json.dump(model_solutions, open('./primary_model_solutions.json', 'w'), indent=4)

In [6]:
s = 0
for model in primary_models:
    s += model.ObjVal
s

7699924.0